In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from poisson_binomial import poisson_binomial_pmf,poisson_binomial_pmf_direct
from scipy import stats,special

In [ ]:
Nt = 8967691

In [2]:
bins = np.array([0.0, 5.0, 16.0, 17.0])
G = 0.5*(bins[1:]+bins[:-1])
G2 = np.square(G[:,np.newaxis]-G[np.newaxis,:])

In [7]:
lg = 3.4
K = np.exp(-G2/2/lg**2)
inv_K = np.linalg.inv(K)
res1 = np.trace(np.dot(inv_K,G2*K/lg**3))
res2 = np.diag(G2).sum()/lg**3

In [9]:
res1,res2

(-0.09893904429423411, 0.0)

# Prior on mu - log prior and gradient

In [ ]:
bins = np.array([0.0, 5.0, 16.0, 17.0])
G = 0.5*(bins[1:]+bins[:-1])
G2 = np.square(G[:,np.newaxis]-G[np.newaxis,:])
M = bins.size-1

mu = np.array([-1.2,3.4,0.3])
m = 1.2
tau2 = 0.3**2.0


def log_prior(mu,m,tau2):
    J = tau2*np.exp(-G2/2.0/m**2)
    J_inv = np.linalg.pinv(J)
    J_inv_mu = np.dot(J_inv,mu)
    
    lnP = +0.5*np.linalg.slogdet(J_inv)[1]-0.5*np.dot(mu.T,J_inv_mu) - (M/2)*np.log(2.0*np.pi)
    
    dJdm = J*G2/m**3
    dJdtau2 = J/tau2

    dlnPdm = -0.5*np.trace(np.dot(J_inv,dJdm))+0.5*np.dot(J_inv_mu.T,np.dot(dJdm,J_inv_mu))
    dlnPdtau2 = -0.5*np.trace(np.dot(J_inv,dJdtau2))+0.5*np.dot(J_inv_mu.T,np.dot(dJdtau2,J_inv_mu))
    dlnPdmu = -J_inv_mu
    
    return lnP, dlnPdmu, dlnPdm, dlnPdtau2

def check(x,y,z):
    print(x,y,(x-y)/z)

lnP, dlnPdmu, dlnPdm, dlnPdtau2 = log_prior(mu,m,tau2)
lnP_truth = stats.multivariate_normal(mean=np.zeros(M),cov=J,allow_singular=True).logpdf(mu)
check(lnP,lnP_truth,lnP_truth)
h = 1e-7

lnP_dmu = log_prior(mu+h*np.array([1,0,0]),m,tau2)[0]
check(lnP+h*dlnPdmu[0],lnP_dmu,lnP_truth)

lnP_dm = log_prior(mu,m+h,tau2)[0]
check(lnP+h*dlnPdm,lnP_dm,lnP_truth)

lnP_dtau2 = log_prior(mu,m,tau2+h)[0]
check(lnP+h*dlnPdtau2,lnP_dtau2,lnP_truth)

# Prior on x

In [183]:
Ng = 10
Nt = 15
x = np.random.randn(Ng*Nt,1)
mu = np.arange(Ng)/10.0
Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))

t = np.arange(Nt)
lt = 10.3
tau = np.abs(t[:,np.newaxis]-t[np.newaxis,:])
K = np.exp(-tau/lt)
invK = np.linalg.pinv(K)
dK_dlt = tau*K/lt/lt
result = np.einsum('ij,jk,kl->il',Y,invK,dK_dlt)
u = np.exp(-1/lt)
u2 = u*u

power_u = np.power(u,np.arange(Nt))

approx = np.zeros((Ng,Nt))

for i in range(Ng):
    for l in range(Nt):
        #approx[i,l] =  (Y[i,0]*np.power(u,l+2) + Y[i,-1]*np.power(u,Nt-l+1) - (1.0+u*u)*Y[i,l])/(1.0-u*u)
        approx[i,l] =  (u2*Y[i,0]*power_u[l] + u2*Y[i,-1]*power_u[Nt-l-1] - (1.0+u2)*Y[i,l])/(1.0-u2)
        for j in range(Nt):
            approx[i,l] += Y[i,j]*power_u[np.abs(l-j)]
approx = approx / (lt*lt)
#print(result)
#print(approx)
np.allclose(result,approx)

True

In [342]:
from numba import njit
from math import ceil

@njit
def func_approx(Ng,Nt,lt,Y,result):
    
    u = np.exp(-1/lt)
    u2 = u*u
    power_u = np.power(u,np.arange(Nt))
    
    lt2 = lt*lt
    
    for i in range(Ng):
        for l in range(Nt):
            res =  (u2*Y[i,0]*power_u[l] + u2*Y[i,-1]*power_u[Nt-l-1] - (1.0+u2)*Y[i,l])/(1.0-u2)
            for j in range(Nt):
                res += Y[i,j]*power_u[abs(l-j)]
            result[i,l] = res/lt2
            
@njit
def func_truncate(Ng,Nt,lt,Y,result,log_eps=np.log(1e-16)):
    
    u = np.exp(-1/lt)
    u2 = u*u
    
    M = 1+ceil(-lt*log_eps)
    
    power_u = np.power(u,np.arange(M))
    
    lt2 = lt*lt
    
    for i in range(Ng):
        
        for l in range(Nt):
            
            res =  - (1.0+u2)*Y[i,l]/(1.0-u2)
            
            if l < M:
                res += u2*Y[i,0]*power_u[l]/(1.0-u2)
                
            if Nt-l-1 < M:
                res += u2*Y[i,-1]*power_u[Nt-l-1]/(1.0-u2)
            
            #res = (u2*( Y[i,0]*power_u[min(M,l)] + Y[i,-1]*power_u[min(M,Nt-l-1)] ) - (1.0+u2)*Y[i,l])/(1.0-u2)
            
            for j in range(max(0,l-M),min(Nt,l+M)):
                res += Y[i,j]*power_u[abs(l-j)]
                
            result[i,l] = res/lt2
            
def func_truth(Ng,Nt,lt,Y,result):
    
    t = np.arange(Nt)
    tau = np.abs(t[:,np.newaxis]-t[np.newaxis,:])
    K = np.exp(-tau/lt)
    invK = np.linalg.pinv(K)
    dK_dlt = tau*K/lt/lt
    result[:] = np.einsum('ij,jk,kl->il',Y,invK,dK_dlt)

In [353]:
Ng = 35
Nt = 30000
x = np.random.randn(Ng*Nt,1)
mu = np.arange(Ng)/10.0
Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))
lt = 10.2
result_approx = np.zeros((Ng,Nt))
result_truncate = np.zeros((Ng,Nt))
result_truth = np.zeros((Ng,Nt))

In [354]:
if Nt < 250:
    %time func_approx(Ng,Nt,lt,Y,result_approx)
    %time func_truncate(Ng,Nt,lt,Y,result_truncate)
    %time func_truth(Ng,Nt,lt,Y,result_truth)
    #%timeit func_approx(Ng,Nt,lt,Y,result_approx)
    print(np.allclose(result_approx,result_truth),np.allclose(result_truncate,result_truth),np.allclose(result_approx,result_truncate))
elif Nt < 30000+1e-10:
    %time func_approx(Ng,Nt,lt,Y,result_approx)
    %time func_truncate(Ng,Nt,lt,Y,result_truncate)
    print(np.allclose(result_approx,result_truncate))
else:
    %time func_truncate(Ng,Nt,lt,Y,result_truncate)

CPU times: user 1min 2s, sys: 599 ms, total: 1min 2s
Wall time: 1min 2s
CPU times: user 1.16 s, sys: 11.2 ms, total: 1.17 s
Wall time: 1.17 s
True


In [339]:
result_truncate

array([[ 0.08710309, -0.05681015,  0.00729948, ..., -0.10865136,
         0.04155819, -0.05323901],
       [ 0.04839903, -0.18979308, -0.05632268, ..., -0.03737061,
        -0.16274004,  0.01148076],
       [-0.05105099,  0.10290865,  0.09382653, ..., -0.06585921,
         0.0741678 ,  0.03547565],
       ...,
       [-0.23365156, -0.08251317, -0.05427989, ..., -0.24204967,
        -0.16892817, -0.14597519],
       [-0.18173588, -0.05047941, -0.22704194, ..., -0.12162819,
        -0.05768479, -0.21454671],
       [-0.16950705, -0.17122475, -0.16266835, ..., -0.16915538,
        -0.31607717, -0.13251984]])

In [283]:
result_truncate

array([[-2.18720904e-04, -4.27300373e-02, -1.21643508e-01, ...,
         1.05849731e-01, -1.37187966e-01, -1.47060371e-01],
       [-5.80000271e-03, -7.43333522e-02, -1.27239138e-01, ...,
         1.00124092e-01,  5.07317459e-02, -1.41964840e-01],
       [ 3.98712167e-02,  1.24647729e-01, -4.10619007e-02, ...,
        -1.47172456e-02, -1.13837435e-01, -5.45681374e-02],
       ...,
       [-1.91659039e-01, -2.45775481e-01, -2.46969719e-01, ...,
        -6.96517647e-02, -6.52374356e-02, -4.09968623e-03],
       [-2.01496463e-01, -1.48768802e-01, -1.77069512e-01, ...,
        -7.09227940e-04, -1.04280434e-01, -3.13293782e-02],
       [-1.54498931e-01, -8.96307892e-02, -7.97093827e-02, ...,
         8.36113795e-02,  4.10608546e-02, -3.88361987e-02]])

(True, False)

In [200]:
result_truth

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [199]:
result_approx

array([[-0.01946621,  0.07082986,  0.10609276,  0.04901776, -0.0972679 ,
         0.01213148, -0.03757926, -0.00733172,  0.04793834, -0.07750527,
        -0.1337639 ,  0.05988404, -0.0600959 , -0.10255377,  0.05842416],
       [-0.03863774,  0.00414771,  0.01570791,  0.03646353,  0.0212866 ,
        -0.20488202,  0.13845745,  0.07788925, -0.11499758,  0.10544695,
         0.20727208,  0.05911946,  0.03389747,  0.02637368, -0.03943151],
       [-0.01718417, -0.08022838, -0.2135082 , -0.03328361, -0.02564966,
        -0.25226414, -0.03566143, -0.0356083 , -0.08127559, -0.04841506,
        -0.11015697, -0.13063095,  0.02194498, -0.04596701,  0.02840173],
       [-0.02533952,  0.03935182,  0.13197137, -0.01040914, -0.11228363,
        -0.12852019, -0.02209008, -0.14851542, -0.12068226,  0.04882719,
         0.08576215, -0.02497621,  0.2358873 , -0.20301883,  0.01795215],
       [ 0.02096934, -0.11402032, -0.06683039, -0.09117287, -0.11270815,
        -0.149073  ,  0.08311172, -0.01414105, 

In [160]:
l=2
i=3
print(np.power(u,np.abs(l-np.arange(Nt))))
print(np.sum(Y[i,:]*np.power(u,np.abs(l-np.arange(Nt)))))

[0.82351398 0.90747671 1.         0.90747671 0.82351398]
-3.0515827081819777


In [166]:
print(power_u)
np.roll(power_u,-l)

[1.         0.90747671 0.82351398 0.74731975 0.67817527]


array([0.82351398, 0.74731975, 0.67817527, 1.        , 0.90747671])

In [153]:
power_u

array([1.        , 0.90747671, 0.82351398, 0.74731975, 0.67817527])

In [91]:
((1-u*u)*np.sum(Y[0,:]*np.power(u,np.arange(Nt)))-Y[0,0]+Y[0,-1]*np.power(u,Nt+1))/ (lt*lt*(1-u*u))

0.005695335314333618

In [9]:
%timeit lg*lg*lg

62.7 ns ± 1.27 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [137]:
l = 5
N = 12
for j in range(2,N-1):
    print(l,j,np.abs(j-1-l),np.abs(j-l),np.abs(j+1-l))

5 2 4 3 2
5 3 3 2 1
5 4 2 1 0
5 5 1 0 1
5 6 0 1 2
5 7 1 2 3
5 8 2 3 4
5 9 3 4 5
5 10 4 5 6


In [357]:
@njit
def func_invKgYinvKt(Ng,Nt,lt,invKgY,invKgYinvKt):

    u = np.exp(-1.0/lt)
    u2 = u*u
    oneplusu2 = 1.0 + u2
    oneoveroneminusu2 = 1.0 / ( 1.0 - u2 )
    
    for ig in range(Ng):
        invKgYinvKt[ig,0] = (invKgY[ig,0] - u * invKgY[ig,1])*oneoveroneminusu2
        invKgYinvKt[ig,-1] = (invKgY[ig,-1] - u * invKgY[ig,-2])*oneoveroneminusu2
        for it in range(1,Nt-1):
            invKgYinvKt[ig,it] = ( oneplusu2 * invKgY[ig,it] - u * ( invKgY[ig,it-1] + invKgY[ig,it+1] ) )*oneoveroneminusu2
            
            
@njit
def func_YJt(Ng,Nt,lt,Y,result,log_eps=np.log(1e-16)):
    
    u = np.exp(-1/lt)
    u2 = u*u
    
    M = 1+ceil(-lt*log_eps)
    
    power_u = np.power(u,np.arange(M))
    
    lt2 = lt*lt
    
    for i in range(Ng):
        
        for l in range(Nt):
            
            res =  - (1.0+u2)*Y[i,l]/(1.0-u2)
            
            if l < M:
                res += u2*Y[i,0]*power_u[l]/(1.0-u2)
                
            if Nt-l-1 < M:
                res += u2*Y[i,-1]*power_u[Nt-l-1]/(1.0-u2)
            
            #res = (u2*( Y[i,0]*power_u[min(M,l)] + Y[i,-1]*power_u[min(M,Nt-l-1)] ) - (1.0+u2)*Y[i,l])/(1.0-u2)
            
            for j in range(max(0,l-M),min(Nt,l+M)):
                res += Y[i,j]*power_u[abs(l-j)]
                
            result[i,l] = res/lt2
    

def log_prior_x(x,mu,sigma2,lt,lg,G2,Ng,Nt):
    
    # X is a (Ng,Nt) matrix
    # mu is a (Ng,) vector
    # sigma2, lt and lg are scalars
    # G2 is a (Ng,Ng) matrix
    
    Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))
    
    Kg = np.exp(-G2/2/lg/lg)
    dKg_dlg = G2*Kg/lg/lg/lg
    Jg = np.linalg.solve(Kg,dKg_dlg)
    invKgY = np.linalg.solve(Kg,Y)
    
    invKgYinvKt = np.zeros((Ng,Nt))
    func_invKgYinvKt(Ng,Nt,lt,invKgY,invKgYinvKt)
    
    logdetKg = np.linalg.slogdet(Kg)[1]
    logdetinvKt = -(Nt-1.0)*np.log( 1.0 - u2 )
    TrJg =  np.trace(Jg)
    TrJt = -2.0*(Nt-1.0)*u2/(1.0-u2)
    
    Y_invKgYinvKt = (Y*invKgYinvKt).sum()
    
    JgTY = np.dot(Jg.T,Y)
    JgTY_invKgYinvKt = (JgTY*invKgYinvKt).sum()
    
    YJt = np.zeros((Ng,Nt))
    func_YJt(Ng,Nt,lt,Y,YJt,log_eps=np.log(1e-16))
    TJt_invKgYinvKt = (YJt*invKgYinvKt).sum()
    
    lnP = -Ng*Nt*np.log(2.0*np.pi*sigma2)/2.0 + Ng*logdetinvKt/2.0 - Nt*logdetKg/2.0 - Y_invKgYinvKt/2.0/sigma2
    
    dlnP_dX = -invKgYinvKt/sigma2
    dlnP_dx = dlnP_dX.ravel()
    dlnP_dmu = -dlnP_dX.sum(axis=1)
    dlnP_dsigma2 = -Ng*Nt/2.0/sigma2 + Y_invKgYinvKt/2.0/sigma2/sigma2
    dlnP_dlt = -Ng*TrJt/2.0 + TJt_invKgYinvKt/2.0/sigma2
    dlnP_dlg = -Nt*TrJg/2.0 + JgTY_invKgYinvKt/2.0/sigma2
    
    return lnP, dlnP_dx, dlnP_dmu, dlnP_dsigma2, dlnP_dlt, dlnP_dlg


In [ ]:

G = 
Ng = 35
Nt = 300
x = np.random.randn(Ng*Nt,1)
mu = np.arange(Ng)/10.0
Y = x.reshape((Ng,Nt)) - mu.reshape((Ng,1))
sigma2, lt, lg = 1.2,10.2,1.3



# Investigating priors on hyperparameters

### Lengthscale

In [ ]:
plt.figure()
x = np.linspace(0.1,10,10001)

# InvGamma(5,5)
plt.plot(x,stats.invgamma(a=1.0,scale=2.0).pdf(x))
plt.yscale('log')

### Variance

In [ ]:
plt.figure()
x = np.linspace(0.1,10,10001)

# InvGamma(5,5)
plt.plot(x,stats.gamma(a=1.0,scale=1.0).pdf(x))
plt.yscale('log')

# Global

In [37]:
stats.rv_discrete(values=(np.arange(stars[i]['n']+1),pmf)).rvs()

0

2

In [5]:
poisson_binomial_pmf_direct(np.array([0.4,0.9,0.96]))

array([0.0024, 0.0808, 0.5712, 0.3456])

In [78]:
a = np.random.randn(3, 3)
b = np.random.randn(3, 1)
c = a*b

In [79]:
c.shape

(3, 3)

In [63]:
#bins = np.array([0.0, 5.0, 16.0, 17.0, 18.0, 18.2, 18.4, 18.6, 18.8, 19.0, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.0, 20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21.0, 21.1, 21.2, 21.3, 21.4, 21.5, 25.0])
bins = np.array([-2.0,-1.0,0.0, 1.0, 2.0])
G = 0.5*(bins[1:]+bins[:-1])
G_squared = np.square(G[:,np.newaxis]-G[np.newaxis,:])
M = bins.size-1
N = 10
p_groundtruth = special.expit(-G[:,np.newaxis]*np.ones((M,N)))
p_groundtruth[:,:3] = 0.0

stars = {}
times = np.arange(N).astype(np.int)
N_star = 123
for i in range(N_star):
    stars[i] = {}
    stars[i]['n'] = np.random.randint(N)
    stars[i]['g'] = np.random.randint(M)
    stars[i]['t'] = np.sort(np.random.choice(times,stars[i]['n'],replace=False))
    probs = p_groundtruth[stars[i]['g'],stars[i]['t']]
    pmf = poisson_binomial_pmf_direct(probs)
    stars[i]['k'] = np.random.choice(np.arange(0, stars[i]['n']+1), p=pmf)
    
#tmp_args = np.concatenate([np.array([4.0,0.5,4.0,0.5]),np.ones(N_bins),np.random.normal(0,1,N_bins*N_times)])

def log_prior_x(x,mu,l,sigma2):
    
    expml = np.exp(-l)
    expm2l = expml*expml
    
    d = x-mu
    
    A = d[0]**2+d[-1]**2
    B = np.sum(d[1:-1]**2)
    C = d[:-1]*d[1:]
    D = d[0] + d[-1]
    E = np.sum(d[1:-1])
    
    lnP = -N/2*np.log(sigma2) - (N-1)/2*np.log(1-expm2l) - N/2*np.log(2*np.pi) - ( A + (1+expm2l)*B - 2*expml*C )/(2*sigma2*(1-expm2l))
    
    dlnPdl = ((N-1)*sigma2*expm2l*(1-expm2l) + C*expml*(1+expm2l) - expm2l*(A+2*B))/(l*l*(1-expm2l)*(1-expm2l))
    
    dlnPdsigma2 = -N/2/sigma2 + ( A + (1+expm2l)*B - 2*expml*C )/(2*sigma2*sigma2*(1-expm2l))
    
    dlnPdmu =  (D+(1-expml)*E)/(sigma2*(1+xpml))
    
    dlnPdx = np.zeros(N)
    dlnPdx[0] = -(d[0]-expml*d[1])/(sigma2*(1-expm2l))
    for i in range(1,N-1):
        dlnPdx[i] = -((1+expm2l)*d[i]-expml*(d[i-1]+d[i+1]))/(sigma2*(1-expm2l))
    dlnPdx[-1] = -(d[-1]-expml*d[-2])/(sigma2*(1-expm2l))
    
    return lnP, dlnPdx, dlnPdmu, dlnPdl, dlnPdsigma2

def log_prior_mu(mu,m,tau2):
    
    J = tau2*np.exp(-G_squared/2.0/m**2)
    J_inv = np.linalg.pinv(J)
    J_inv_mu = np.dot(J_inv,mu)
    
    lnQ = +0.5*np.linalg.slogdet(J_inv)[1]-0.5*np.dot(mu.T,J_inv_mu) - (M/2)*np.log(2.0*np.pi)
    
    dJdm = J*G2/m**3
    dJdtau2 = J/tau2

    dlnQdm = -0.5*np.trace(np.dot(J_inv,dJdm))+0.5*np.dot(J_inv_mu.T,np.dot(dJdm,J_inv_mu))
    dlnQdtau2 = -0.5*np.trace(np.dot(J_inv,dJdtau2))+0.5*np.dot(J_inv_mu.T,np.dot(dJdtau2,J_inv_mu))
    dlnQdmu = -J_inv_mu
    
    return lnQ, dlnQdmu, dlnQdm, dlnQdtau2

def log_prior_lengthscale(l):
    
    # InvGamma(1,2)
    
    lnA = np.log(2.0)-2.0*np.log(l)-2.0/l
    
    dlnAdl = 2.0*(1.0-l)/l/l
    
    return lnA, dlnAdl

def log_prior_variance(sigma2):
    
    # Gamma(1,1)
    
    lnS = - sigma2
    
    dlnSdsigma2 = - 1.0
    
    return lnS, dlnSsigma2

def log_likelihood(p):
    lnL, dlnLdp = 0.0, np.zeros(M*N)
    
    for star in stars:
        L, dLdp = 0,0

def log_posterior(args):
    
    # Unpack parameters
    lnl, lnsigma2, lnm, lntau2 = args[:4]
    mu = args[4:4+M]
    x = args[4+M:]
    
    # Transform parameters
    l, sigma2, m, tau2 = np.exp(lnl), np.exp(lnsigma2), np.exp(lnm), np.exp(lntau2)
    p = 1.0/(1.0+np.exp(-x))
    
    ##### Priors
    
    ### Initialise values
    
    lnF = 0.0
    dlnFdl ,dlnFdm = 0.0, 0.0
    dlnFdsigma2, dlnFdtau2 = 0.0, 0.0
    dlnFdmu = np.zeros(M)
    dlnFdx = np.zeros(M*N)
    dlnFdp = np.zeros(M*N)
    
    
    ### Prior on x
    
    for g in range(M):
        
        # Calculate quantities
        lnP, dlnPdx, dlnPdmu, dlnPdl, dlnPdsigma2 = log_prior_x(x[g*N:(g+1)*N],mu[g],l,sigma2)
        
        # Increment log posterior
        lnF += lnP
        
        # Increment gradients
        dlnFdx[g*N:(g+1)*N] += dlnPdx
        dlnFdmu[g] += dlnPdmu
        dlnFdl += dlnPdl
        dlnFdsigma2 += dlnPdsigma2
    
    ### Prior on mu
    
    # Calculate quantities
    lnQ, dlnQdmu, dlnQdm, dlnQdtau2 = log_prior_mu(mu,m,tau2)
    
    # Increment log posterior
    lnF += lnQ
    
    # Increment gradients
    dlnFdmu += dlnQdmu
    dlnFdm += dlnQdm
    dlnFdtau2 += dlnQdtau2
    
    
    ### Prior on l and m
    
    # Calculate quantities
    lnA, dlnAdl = log_prior_lengthscale(l)
    lnB, dlnBdm = log_prior_lengthscale(m)
    
    # Increment log posterior
    lnF += lnA
    lnF += lnB
    
    # Increment gradients
    dlnFdl += dlnAdl
    dlnFdm += dlnBdm
    
    ### Prior on sigma2 and tau2
    
    # Calculate quantities
    lnS, dlnSdsigma2 = log_prior_variance(sigma2)
    lnT, dlnTdtau2 = log_prior_variance(tau2)
    
    # Increment log posterior
    lnF += lnS
    lnF += lnT
    
    # Increment gradients
    dlnFdsigma2 += dlnSdsigma2
    dlnFdtau2 += dlnTdtau2
    
    ##### Likelihood
    
    # Calculate quantities
    lnL, dlnLdp = log_likelihood(p)
        
    # Increment log posterior
    lnF += lnL
    
    # Increment gradients
    dlnFdp += dlnLdp
    
    ##### Construct gradient with respect to input parameters
    
    # Initialise
    dlnFdargs = np.zeros(4+M+M*N)
    
    # Correct for log-parameterisation
    dlnFdlnl = l*dlnFdl
    dlnFdlnm = m*dlnFdm
    dlnFdlnsigma2 = sigma2*dlnFdsigma2
    dlnFdlntau2 = tau2*dlnFdtau2
    
    # Correct for logit-parameterisation
    dlnFdx += (1.0/p+1.0/(1.0-p))*dlnFdp
    
    # Fill in values
    dlnFdargs[0] = dlnFdlnl
    dlnFdargs[1] = dlnFdlnsigma2
    dlnFdargs[2] = dlnFdlnm
    dlnFdargs[3] = dlnFdlntau2
    dlnFdargs[4:4+M] = dlnFdmu
    dlnFdargs[4+M:] = dlnFdx
    
    return lnF, dlnFdargs
    

SyntaxError: invalid syntax (<ipython-input-63-2fe86dc56c0d>, line 94)

# Poisson binomial likelihood

In [72]:
p = np.array([0.3,0.6,0.2,0.1,0.9,0.5])
n = p.size
pmf = np.zeros(n+1)
poisson_binomial_pmf(p,n,pmf)
print(pmf,pmf.sum())
print(poisson_binomial_pmf_direct(p))

[0.01008 0.12388 0.3353  0.3484  0.153   0.02772 0.00162] 1.0
[0.01008 0.12388 0.3353  0.3484  0.153   0.02772 0.00162]


In [66]:
from numba import njit
            
@njit
def poisson_binomial_likelihood(k,probs,probslen,pmf,subpmf,likelihood,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood[0] = pmf[k]
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k]
        
    

k = 5
subpmf = np.zeros(n)
gradient = np.zeros(n)
likelihood = np.zeros(1)
poisson_binomial_likelihood(k,p,n,pmf,subpmf,likelihood,gradient)

dp = 1e-4
print(likelihood[0])
print(poisson_binomial_pmf_easy(p+dp*np.array([0,0,0,1,0,0]))[k])
print(likelihood[0]+dp*gradient[3])

0.027719999999999998
0.02773152
0.02773152


In [ ]:
@njit
def poisson_binomial_log_likelihood(k,probs,probslen,pmf,subpmf,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood = pmf[k]
    log_likelihood = np.log(likelihood)
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k]
        
    return log_likelihood

In [ ]:
dp = 1e-7*np.random.uniform(-1,1,6)
print(likelihood[0])
print(poisson_binomial_pmf_easy(p+dp)[k])
print(likelihood[0]+np.dot(dp,gradient))

In [ ]:
from numba import njit
            
@njit
def poisson_binomial_log_likelihood(k,probs,probslen,pmf,subpmf,log_likelihood,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood = pmf[k]
    log_likelihood[0] = np.log(likelihood)
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = (gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k])/likelihood
        
    

k = 4
subpmf = np.zeros(n)
gradient = np.zeros(n)
log_likelihood = np.zeros(1)
poisson_binomial_log_likelihood(k,p,n,pmf,subpmf,log_likelihood,gradient)

dp = 1e-7*np.random.uniform(-1,1,6)
print(log_likelihood[0])
print(np.log(poisson_binomial_pmf_easy(p+dp)[k]))
print(log_likelihood[0]+np.dot(dp,gradient))

In [24]:
dKt_dl = tau*K/l/l
print(np.trace(np.dot(K_inv,dKt_dl)))
print(-2*u*u*(N-1)/(1-u*u))
    

-1.2521411419973258
-1.2521411419973254


In [23]:
dKt_dl,u

(array([[0.        , 0.36787944, 0.27067057, 0.14936121, 0.07326256],
        [0.36787944, 0.        , 0.36787944, 0.27067057, 0.14936121],
        [0.27067057, 0.36787944, 0.        , 0.36787944, 0.27067057],
        [0.14936121, 0.27067057, 0.36787944, 0.        , 0.36787944],
        [0.07326256, 0.14936121, 0.27067057, 0.36787944, 0.        ]]),
 0.36787944117144233)

In [19]:
N = 5
x = np.arange(N)
l = 1.0
tau = np.abs(x[:,np.newaxis]-x[np.newaxis,:])
K = np.exp(-tau/l)
K_inv = np.linalg.pinv(K)
K_inv[np.abs(K_inv) < 1e-14] = 0.0
print(K_inv)
print(np.linalg.det(K_inv))
u = np.exp(-1.0/l)
a = 1
b = 1+u*u
c = -u
#print(a,b,c)
J_inv = np.zeros(K.shape)
J_inv[0,0] = J_inv[-1,-1] = a
for i in range(1,N-1):
    J_inv[i,i] = b
for i in range(0,N-1):
    J_inv[i,i+1] = J_inv[i+1,i] = c
J_inv /= 1-u*u
print(J_inv)
print(np.power(1/(1-u*u),N-1))

[[ 1.15651764 -0.42545906  0.          0.          0.        ]
 [-0.42545906  1.31303529 -0.42545906  0.          0.        ]
 [ 0.         -0.42545906  1.31303529 -0.42545906  0.        ]
 [ 0.          0.         -0.42545906  1.31303529 -0.42545906]
 [ 0.          0.          0.         -0.42545906  1.15651764]]
1.7889946812194075
[[ 1.15651764 -0.42545906  0.          0.          0.        ]
 [-0.42545906  1.31303529 -0.42545906  0.          0.        ]
 [ 0.         -0.42545906  1.31303529 -0.42545906  0.        ]
 [ 0.          0.         -0.42545906  1.31303529 -0.42545906]
 [ 0.          0.          0.         -0.42545906  1.15651764]]
1.788994681219407


In [102]:
slinalg.cholesky(G)

LinAlgError: 1-th leading minor of the array is not positive definite

In [99]:
G = np.exp(-tau/l)*tau/l/l
G_inv = np.linalg.pinv(G)
print(G_inv)

[[ 3.92358517  4.59283433  0.79863201 -3.23369341 -5.77084919]
 [ 4.59283433  0.81199902  0.77154032 -1.81199902 -3.23369341]
 [ 0.79863201  0.77154032 -1.69054892  0.77154032  0.79863201]
 [-3.23369341 -1.81199902  0.77154032  0.81199902  4.59283433]
 [-5.77084919 -3.23369341  0.79863201  4.59283433  3.92358517]]


In [29]:
np.linalg.inv(np.linalg.cholesky(G)).T

array([[1.        , 0.36787944, 0.13533528, 0.04978707, 0.01969691],
       [0.        , 1.        , 0.36787944, 0.13533528, 0.05354177],
       [0.        , 0.        , 1.        , 0.36787944, 0.14554161],
       [0.        , 0.        , 0.        , 1.        , 0.39562311],
       [0.        , 0.        , 0.        , 0.        , 1.0754151 ]])

In [48]:
def print_matrix(X):
    X[np.abs(X)<1e-14] = 0.0
    print(X)
print_matrix(np.linalg.cholesky(np.linalg.pinv(K)))
print_matrix(np.linalg.pinv(np.linalg.cholesky(K)))

[[ 1.0754151   0.          0.          0.          0.        ]
 [-0.39562311  1.0754151   0.          0.          0.        ]
 [ 0.         -0.39562311  1.0754151   0.          0.        ]
 [ 0.          0.         -0.39562311  1.0754151   0.        ]
 [ 0.          0.          0.         -0.39562311  1.        ]]
[[ 1.          0.          0.          0.          0.        ]
 [-0.39562311  1.0754151   0.          0.          0.        ]
 [ 0.         -0.39562311  1.0754151   0.          0.        ]
 [ 0.          0.         -0.39562311  1.0754151   0.        ]
 [ 0.          0.          0.         -0.39562311  1.0754151 ]]


In [60]:
from scipy import linalg as slinalg

In [89]:
slinalg.cholesky(inv_X)

array([[ 0.77459667, -0.25819889],
       [ 0.        ,  0.57735027]])

In [96]:
np.linalg.cholesky(K_inv)

array([[ 1.0754151 ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.39562311,  1.0754151 ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.39562311,  1.0754151 ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.39562311,  1.0754151 ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.39562311,  1.        ]])

In [84]:
X = np.array([[2,1],[1,3]])
inv_X = np.linalg.inv(X)
chol_X = np.linalg.cholesky(X)
chol_inv_X = np.linalg.cholesky(inv_X)
inv_chol_X = np.linalg.inv(chol_X)
psuedo_chol_inv_X = inv_chol_X.T
print(X)
print(np.dot(chol_X,chol_X.T))
print(inv_X)
print(np.dot(chol_inv_X,chol_inv_X.T))
print(inv_chol_X)
print(chol_inv_X)
print(np.dot(psuedo_chol_inv_X,psuedo_chol_inv_X.T))
print(np.dot(chol_inv_X,chol_inv_X.T))

[[2 1]
 [1 3]]
[[2. 1.]
 [1. 3.]]
[[ 0.6 -0.2]
 [-0.2  0.4]]
[[ 0.6 -0.2]
 [-0.2  0.4]]
[[ 0.70710678  0.        ]
 [-0.31622777  0.63245553]]
[[ 0.77459667  0.        ]
 [-0.25819889  0.57735027]]
[[ 0.6 -0.2]
 [-0.2  0.4]]
[[ 0.6 -0.2]
 [-0.2  0.4]]


In [88]:
inv_chol_X

array([[ 0.70710678,  0.        ],
       [-0.31622777,  0.63245553]])

In [69]:
np.dot(chol_inv_X,chol_inv_X.T)

array([[ 0.6, -0.2],
       [-0.2,  0.4]])

In [70]:
np.dot(inv_chol_X.T,inv_chol_X)

array([[ 0.6, -0.2],
       [-0.2,  0.4]])

In [63]:
slinalg.inv(slinalg.cholesky(X).T)

array([[ 0.70710678, -0.        ],
       [-0.40824829,  0.81649658]])

In [64]:
slinalg.cholesky(slinalg.inv(X)).T

array([[ 0.81649658,  0.        ],
       [-0.40824829,  0.70710678]])

In [ ]:
def true_det(i):
    return np.linalg.det(K_inv[:i,:i])

u = np.exp(1.0/l)
i = 6
print(np.power(u*u/(u*u-1),i))
print(true_det(i))
print(c*true_det(i-1)-a*a*true_det(i-2))

In [ ]:
1-1.0/(u*u)

In [ ]:
K_inv[:1+1,:1+1]

In [ ]:
np.linalg.det(K_inv[:5,:5])

In [ ]:
-np.exp(1.0/l)/(np.exp(2.0/l)-1)

In [ ]:
np.exp(2.0/l)/(np.exp(2.0/l)-1)